In [1]:
# Required columns:  
#   -- Location
#   -- Title
#   -- Salary
#   -- Job Description
#   -- Major Requirements
#   -- Skills
#   -- Experience Level
#   -- Job Type: Full-time, Part-time, Remote, Contract, etc.

In [2]:
import os
from google.colab import drive

In [3]:
# Mount the drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
INPUT_FOLDER = '/content/gdrive/Shareddrives/CSE 144 DATA/DATA'
os.listdir(INPUT_FOLDER)

['cc.en.300.bin',
 'Job Descriptions Dataset 2.csv',
 'gauravduttakiit_resume-dataset.csv',
 'snehaanbhawal_resume-dataset.csv',
 'Job Descriptions Dataset 1.csv',
 'Final Job Dataset.csv',
 'Clean Resume Guara.csv']

In [5]:
!pip install pandas
!pip install re
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
import re
import nltk

In [7]:
# Read Datasets
ds1 = pd.read_csv(INPUT_FOLDER + '/Job Descriptions Dataset 1.csv', encoding='utf-8')
ds2 = pd.read_csv(INPUT_FOLDER + '/Job Descriptions Dataset 2.csv', encoding='utf-8')

### Drop DS1 columns

In [8]:
ds1.head(1)

,Job Title,Job Description,Job Type,Categories,Location,City,State,Country,Zip Code,Address,...,Employer Phone,Employer Logo,Companydescription,Employer Location,Employer City,Employer State,Employer Country,Employer Zip Code,Uniq Id,Crawl Timestamp
0,Shift Manager,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,NaN,"Mission Hills, CA 91345",Mission Hills,CA,United States,91345,NaN,...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,Del Taco is an American quick service restaura...,"Mission Hills, CA 91345",Mission Hills,CA,United States,91345,511f9a53920f4641d701d51d3589349f,2019-08-24 09:13:18 +0000


In [9]:
ds1.columns

Index(['Job Title', 'Job Description', 'Job Type', 'Categories', 'Location',
       'City', 'State', 'Country', 'Zip Code', 'Address', 'Salary From',
       'Salary To', 'Salary Period', 'Apply Url', 'Apply Email', 'Employees',
       'Industry', 'Company Name', 'Employer Email', 'Employer Website',
       'Employer Phone', 'Employer Logo', 'Companydescription',
       'Employer Location', 'Employer City', 'Employer State',
       'Employer Country', 'Employer Zip Code', 'Uniq Id', 'Crawl Timestamp'],
      dtype='object')

In [10]:
# Drop columns
drop_columns_ds1 = ['Crawl Timestamp', 'Uniq Id', 'Employer Zip Code', 
                    'Employer Country', 'Employer State', 'Employer Location', 
                    'Employer City', 'Employer Phone', 'Employer Logo', 
                    'Companydescription', 'Employer Email', 'Employer Website',
                    'Industry', 'Apply Url', 'Apply Email', 'Employees', 
                    'Salary To', 'Salary Period', 'Address', 'Salary From', 
                    'Categories']

modified_ds1 = ds1.drop(drop_columns_ds1, axis=1)

In [11]:
modified_ds1.head(1)

,Job Title,Job Description,Job Type,Location,City,State,Country,Zip Code,Company Name
0,Shift Manager,"<div id=""jobDescriptionText"" class=""jobsearch-...",NaN,"Mission Hills, CA 91345",Mission Hills,CA,United States,91345,Del Taco LLC


### Clean Modified DS1

In [12]:
modified_ds1['Job Description'][0]

'<div id="jobDescriptionText" class="jobsearch-jobDescriptionText">WE ARE LOOKING FOR TOP PERFORMERS TO GROW WITH US!\n<br><br>\nDel Taco needs people with the ability to handle multiple responsibilities and deliver the quality and guest service that defines our brand. While we look for you to bring the right skills and personality for a Shift Manager position, we\'ll also provide the training and support you need to continue to grow and advance.\n<br><br>\nOur Shift Managers support our stores and General Managers in creating an amazing guest experience, and empowering their team to be at their very best. Our Shift Managers create a culture in our restaurants that is exciting, optimistic and rewarding.\n<br><br>\nDel Taco Shift Managers have significant opportunities for growth and career advancement and are charged with supporting the General Manager with the tasks of training the next generation of leaders, guiding them to be promoted and empowering them to have a successful career.

In [13]:
import nltk
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
stop_words = set(stopwords.words('english'))  # Set of English stopwords

In [16]:
modified_ds1['Job Description'][0]

'<div id="jobDescriptionText" class="jobsearch-jobDescriptionText">WE ARE LOOKING FOR TOP PERFORMERS TO GROW WITH US!\n<br><br>\nDel Taco needs people with the ability to handle multiple responsibilities and deliver the quality and guest service that defines our brand. While we look for you to bring the right skills and personality for a Shift Manager position, we\'ll also provide the training and support you need to continue to grow and advance.\n<br><br>\nOur Shift Managers support our stores and General Managers in creating an amazing guest experience, and empowering their team to be at their very best. Our Shift Managers create a culture in our restaurants that is exciting, optimistic and rewarding.\n<br><br>\nDel Taco Shift Managers have significant opportunities for growth and career advancement and are charged with supporting the General Manager with the tasks of training the next generation of leaders, guiding them to be promoted and empowering them to have a successful career.

In [17]:
html_regex = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

# Function from: https://www.kaggle.com/code/gauravduttakiit/resume-screening-using-machine-learning
def cleanJobDescription(text):
    text = re.sub('http\S+\s*', ' ', text)                                                     # remove URLs
    text = re.sub(html_regex, '', text)                                                        # remove HTML tags
    text = re.sub('RT|cc', ' ', text)                                                          # remove RT and cc
    text = re.sub('#\S+', '', text)                                                            # remove hashtags
    text = re.sub('@\S+', '  ', text)                                                          # remove mentions
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', text)       # remove punctuations
    text = re.sub(r'[^\x00-\x7f]',r' ', text) 
    text = re.sub('\s+', ' ', text)                                                            # remove extra whitespace

    tokens = nltk.word_tokenize(text)                                                          # Tokenize the text into words
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]              # Remove stopwords
    filtered_text = ' '.join(filtered_tokens)                                                  # Join the filtered tokens back into text

    return filtered_text.replace('\n', ' ')

In [18]:
modified_ds1['Job Description'] = modified_ds1['Job Description'].apply(lambda x: cleanJobDescription(x))

In [19]:
modified_ds1['Job Description'][0]

'LOOKING TOP PERFORMERS GROW US Del Taco needs people ability handle multiple responsibilities deliver quality guest service defines brand look bring right skills personality Shift Manager position also provide training support need continue grow advance Shift Managers support stores General Managers creating amazing guest experience empowering team best Shift Managers create culture restaurants exciting optimistic rewarding Del Taco Shift Managers significant opportunities growth career advancement charged supporting General Manager tasks training next generation leaders guiding promoted empowering su essful career Essential Job Functions Supports General Manager supervising team delivers effective results areas quality guest service food cost labor cost safety security cleanliness product preparation Running shifts supporting GM managing execution Company policies procedures programs systems Ensures objectives reached areas following Company guidelines Ensure compliance shifts federa

### Drop DS2 Columns

In [20]:
ds2.head(1)

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e


In [21]:
ds2.columns

Index(['country', 'country_code', 'date_added', 'has_expired', 'job_board',
       'job_description', 'job_title', 'job_type', 'location', 'organization',
       'page_url', 'salary', 'sector', 'uniq_id'],
      dtype='object')

In [22]:
# Drop columns
drop_columns_ds2 = ['country_code', 'date_added', 'has_expired', 'job_board',
                    'page_url', 'uniq_id', 'salary']
modified_ds2 = ds2.drop(drop_columns_ds2, axis=1)

# Rename columns
rename_dict_ds2 = {'country': 'Country', 'job_description': 'Job Description', 
                   'job_type': 'Job Type', 'location': 'Location', 
                   'organization': 'Company Name',
                   'sector': 'Sector', 'job_title': 'Job Title'}
modified_ds2 = modified_ds2.rename(columns=rename_dict_ds2)

In [23]:
modified_ds2.head(1)

,Country,Job Description,Job Title,Job Type,Location,Company Name,Sector
0,United States of America,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,IT/Software Development


In [24]:
import re

# Make City, State, Zip Code Columns from Location
def extract_csz(location):
    return re.findall('^([^,]+),\s([A-Z]{2})(?:\s(\d{5}))?$', location)

In [25]:
# modified_ds2['City'], modified_ds2['State'], modified_ds2['Zip Code'] = extract_csz(modified_ds1['Location'])[0]
city_col, state_col, zip_col = [], [], []

for location in modified_ds2['Location']:
  res = extract_csz(location)
  city, state, zip = extract_csz(location)[0] if len(res) == 1 else ('', '', '')
  city_col.append(city)
  state_col.append(state)
  zip_col.append(zip)

modified_ds2['City'], modified_ds2['State'], modified_ds2['Zip Code'] = city_col, state_col, zip_col
city_col, state_col, zip_col = [], [], []

In [26]:
modified_ds2.head(1)

,Country,Job Description,Job Title,Job Type,Location,Company Name,Sector,City,State,Zip Code
0,United States of America,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,IT/Software Development,Madison,WI,53702


### Clean DS2

In [27]:
modified_ds2['Job Description'][0]

"TeamSoft is seeing an IT Support Specialist to join our client in Madison, WI. The ideal candidate must have at least 6 years of experience in the field. They need to be familiar with a variety of the field's concepts, practices, and procedures as this position relies on extensive experience and judgment to plan and accomplish goals. Required Skills:• Call tracking software• Phone based technical support• Problem documentation and communication• Remote Desktop Management Tools• Respond to customer requests• General understanding of LANDesk• Microsoft Office 2007 SuiteFind out why TeamSoft is the Madison area's technology leader with over 200 IT consultants. Owned, operated, and managed by IT consultants, TeamSoft is the clear choice for your career. Join the TeamSoft family today!Did you know?• TeamSoft offers salaried options for many positions that include 5 weeks PTO and great benefits!• TeamSoft has been in business since 1996• TeamSoft's owners are local IT professionals who poss

In [28]:
modified_ds2['Job Description'] = modified_ds2['Job Description'].apply(lambda x: cleanJobDescription(x))

In [29]:
modified_ds2['Job Description'][0]

'TeamSoft seeing Support Specialist join client Madison WI ideal candidate must least 6 years experience field need familiar variety field concepts practices procedures position relies extensive experience judgment plan omplish goals Required Skills Call tracking software Phone based technical support Problem documentation communication Remote Desktop Management Tools Respond customer requests General understanding LANDesk Microsoft Office 2007 SuiteFind TeamSoft Madison area technology leader 200 consultants Owned operated managed consultants TeamSoft clear choice career Join TeamSoft family today know TeamSoft offers salaried options many positions include 5 weeks PTO great benefits TeamSoft business since 1996 TeamSoft owners local professionals possess wealth experience application development business analysis project management TeamSoft Recruiters knowledgeable friendly ready help make next great career move TeamSoft offers full benefits suite W2 consultants including 401k plan 1

### Combine both datasets

In [30]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import numpy as np

In [32]:
print('Shared columns : ', np.intersect1d(modified_ds2.columns, modified_ds1.columns))
print('Unique columns : ', list(set(modified_ds1.columns) ^ set(modified_ds2.columns)))

Shared columns :  ['City' 'Company Name' 'Country' 'Job Description' 'Job Title' 'Job Type'
 'Location' 'State' 'Zip Code']
Unique columns :  ['Sector']


In [33]:
modified_ds1['Sector'] = ''
modified_ds1['Job Type'] = ''

In [34]:
print('Unique columns : ', list(set(modified_ds1.columns) ^ set(modified_ds2.columns)))

Unique columns :  []


In [35]:
modified_ds1.dtypes

Job Title          object
Job Description    object
Job Type           object
Location           object
City               object
State              object
Country            object
Zip Code           object
Company Name       object
Sector             object
dtype: object

In [36]:
modified_ds2.dtypes

Country            object
Job Description    object
Job Title          object
Job Type           object
Location           object
Company Name       object
Sector             object
City               object
State              object
Zip Code           object
dtype: object

In [37]:
combined_ds = pd.concat([modified_ds1, modified_ds2], axis=0).reset_index(drop=True)

In [38]:
combined_ds.head(1)

,Job Title,Job Description,Job Type,Location,City,State,Country,Zip Code,Company Name,Sector
0,Shift Manager,LOOKING TOP PERFORMERS GROW US Del Taco needs ...,,"Mission Hills, CA 91345",Mission Hills,CA,United States,91345,Del Taco LLC,


### From Empty to NaN

In [39]:
combined_ds['Job Type'][0]

''

In [40]:
combined_ds.replace('', np.nan, inplace=True)
combined_ds.replace('NaN', np.nan, inplace=True)

In [41]:
combined_ds['Job Type'][0]

nan

### Save to a CSV

In [42]:
combined_ds.to_csv(INPUT_FOLDER + '/Final Job Dataset.csv', index=False)